<a href="https://colab.research.google.com/github/Dahuginn/bewise_parcing_test/blob/main/%5BMosina%5Dbewise_parcing_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pymorphy2
!pip install yargy

In [2]:
import pandas as pd
import numpy as np
import pymorphy2
from yargy import Parser, rule, or_
from yargy.predicates import gram, dictionary
from yargy.pipelines import morph_pipeline

# Загрузка данных

In [3]:
test_data = pd.read_csv('test_data.csv')

In [4]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 480 entries, 0 to 479
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   dlg_id  480 non-null    int64 
 1   line_n  480 non-null    int64 
 2   role    480 non-null    object
 3   text    480 non-null    object
dtypes: int64(2), object(2)
memory usage: 15.1+ KB


In [5]:
test_data.head()

,dlg_id,line_n,role,text
0,0,0,client,Алло
1,0,1,manager,Алло здравствуйте
2,0,2,client,Добрый день
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес з...
4,0,4,client,Ага


Создадим колонку и будем помечать флажками, если нашли, что искали

In [7]:
test_data['gave_name'] = " "
test_data['name'] = " "
test_data['greeting'] = " "
test_data['company'] = " "
test_data['bye'] = " "

Нам нужны только реплики менеджера:

In [8]:
text = test_data[test_data['role'] == 'manager']

# Извлекаем реплики с приветствием

In [9]:
greeting = morph_pipeline(['здравствуйте', 'добрый день'])
rule_greeting = rule(greeting)
parser = Parser(rule_greeting)
for index, row in text.iterrows():
  for match in parser.findall(row['text']):
    test_data.at[index,'greeting'] = True

# Извлекаем реплики, где менеджер представился

Когда менеджер представляется, он говорит "меня зовут <>" или "<> меня зовут", "это <>".

In [10]:
words = {'зовут', 'это'}
rule_name = or_(rule(dictionary(words), gram('Name')), rule(gram('Name'), dictionary({'зовут', 'это'})))
parser = Parser(rule_name)
for index, row in text.iterrows():
  for match in parser.findall(row['text']):
    test_data.at[index,'gave_name'] = True
    for i in [_.value for _ in match.tokens]:
      if i not in words:
          test_data.at[index,'name'] = i

# Извлекаем название компании

После представления менеджер называет имя компании. Менеджер говорит "компания <название компании>"

In [11]:
words = {'компания', 'компании'}
rule_name = rule(dictionary(words), gram('NOUN'))
parser = Parser(rule_name)
for index, row in text.iterrows():
  for match in parser.findall(row['text']):
    for i in [_.value for _ in match.tokens]:
      if i != 'компания':
        test_data.at[index,'company'] = i

# Извлекаем, где менеджер попрощался

In [12]:
bye = morph_pipeline(['до свидания', 'всего доброго'])
rule_bye = rule(bye)
parser = Parser(rule_bye)
for index, row in text.iterrows():
  for match in parser.findall(row['text']):
    test_data.at[index,'bye'] = True

# Проверка менеджера

Так выглядят данные после парсинга (на примере первого диалога):

In [16]:
test_data[test_data['dlg_id'] == 0]

,dlg_id,line_n,role,text,gave_name,name,greeting,company,bye
0,0,0,client,Алло,,,,,
1,0,1,manager,Алло здравствуйте,,,True,,
2,0,2,client,Добрый день,,,,,
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес з...,True,ангелина,,диджитал,
4,0,4,client,Ага,,,,,
...,...,...,...,...,...,...,...,...,...
104,0,104,client,Никто вот,,,,,
105,0,105,client,Да спасибо огромное,,,,,
106,0,106,manager,И вам спасибо большое за обратную связь,,,,,
107,0,107,client,Угу,,,,,


Проверим условие «В каждом диалоге обязательно необходимо поздороваться и попрощаться с клиентом». Вернем название диалога и True, если требование выполнено, False - в обратном случае.

In [13]:
def good_manager(data):
  if (data.loc[data['greeting'] == True].empty == True):
    return (np.unique(data['dlg_id'])[0], False)
  elif data.loc[data['bye'] == True].empty == True:
    return (np.unique(data['dlg_id'])[0], False)
  else:
    return (np.unique(data['dlg_id'])[0], True)

Отберем все диалоги:

In [14]:
dgs = [test_data[test_data['dlg_id'] == 0], test_data[test_data['dlg_id'] == 1], test_data[test_data['dlg_id'] == 2], test_data[test_data['dlg_id'] == 3], test_data[test_data['dlg_id'] == 4], test_data[test_data['dlg_id'] == 5]]

In [15]:
for i in dgs:
  num, is_good = good_manager(i)
  print(f"Номер диалога: {num}, требование выполнено: {is_good}")

Номер диалога: 0, требование выполнено: True
Номер диалога: 1, требование выполнено: True
Номер диалога: 2, требование выполнено: False
Номер диалога: 3, требование выполнено: True
Номер диалога: 4, требование выполнено: False
Номер диалога: 5, требование выполнено: False
